In [ ]:
import pandas as pd
import numpy as np

In [ ]:
merged = pd.read_parquet('processed_data/merged2.parquet')

In [3]:
schedule = pd.read_parquet('processed_data/schedule.parquet')

In [138]:
merged.shape

(154643, 176)

#### все колонки для удобного копирования из файла

In [7]:
merged.columns.to_series().to_csv('metadata/merged_columns.csv')

### Убираем тех, о ком нет информации о сыгранных минутах

In [137]:
merged = merged[~merged.loc[:, ('summary', 'min', 'Unnamed: 9_level_1')].isna()]

### убираем тех, кто сыграл меньше 45 минут

In [139]:
# сколько тех, кто сыграл меньше 45 минут
(merged.loc[:, ('summary', 'min', 'Unnamed: 9_level_1')] <= 45).sum()

np.int64(46521)

In [140]:
#удаляем их
merged = merged[~(merged.loc[:, ('summary', 'min', 'Unnamed: 9_level_1')] <= 45)]

### Сохраняем изменённый merged

In [132]:
merged.to_parquet('processed_data/merged2.parquet')

### убираем, где много пустых

In [ ]:
# не берём в расчёт вратарей
keepers = merged.loc[:, ('summary', 'pos', 'Unnamed: 7_level_1')] == 'GK' #булевый вектор
more60_is_zero = (merged == 0).sum(axis = 1).sort_values(ascending=False) > 60
merged = merged.loc[keepers | ~keepers & ~more60_is_zero]

### Удаляем дубликаты

In [156]:
# Проверяем, есть ли дубликаты строк
duplicates_rows = merged.duplicated()
merged[duplicates_rows == True]

Empty DataFrame
Columns: [(summary, jersey_number, Unnamed: 5_level_1), (summary, nation, Unnamed: 6_level_1), (summary, pos, Unnamed: 7_level_1), (summary, age, Unnamed: 8_level_1), (summary, min, Unnamed: 9_level_1), (summary, Performance, Gls), (summary, Performance, Ast), (summary, Performance, PK), (summary, Performance, PKatt), (summary, Performance, Sh), (summary, Performance, SoT), (summary, Performance, CrdY), (summary, Performance, CrdR), (summary, Performance, Touches), (summary, Performance, Tkl), (summary, Performance, Int), (summary, Performance, Blocks), (summary, Expected, xG), (summary, Expected, npxG), (summary, Expected, xAG), (summary, SCA, SCA), (summary, SCA, GCA), (summary, Passes, Cmp), (summary, Passes, Att), (summary, Passes, Cmp%), (summary, Passes, PrgP), (summary, Carries, Carries), (summary, Carries, PrgC), (summary, Take-Ons, Att), (summary, Take-Ons, Succ), (summary, game_id, Unnamed: 35_level_1), (defense, jersey_number, Unnamed: 5_level_1), (defense, nation, Unnamed: 6_level_1), (defense, pos, Unnamed: 7_level_1), (defense, age, Unnamed: 8_level_1), (defense, Tackles, min), (defense, Tackles, Tkl), (defense, Tackles, TklW), (defense, Tackles, Def 3rd), (defense, Tackles, Mid 3rd), (defense, Tackles, Att 3rd), (defense, Challenges, Tkl), (defense, Challenges, Att), (defense, Challenges, Tkl%), (defense, Challenges, Lost), (defense, Blocks, Blocks), (defense, Blocks, Sh), (defense, Blocks, Pass), (defense, Int, Unnamed: 22_level_1), (defense, Tkl+Int, Unnamed: 23_level_1), (defense, Clr, Unnamed: 24_level_1), (defense, Err, Unnamed: 25_level_1), (defense, game_id, Unnamed: 26_level_1), (keepers, nation, Unnamed: 5_level_1), (keepers, age, Unnamed: 6_level_1), (keepers, min, Unnamed: 7_level_1), (keepers, Shot Stopping, SoTA), (keepers, Shot Stopping, GA), (keepers, Shot Stopping, Saves), (keepers, Shot Stopping, Save%), (keepers, Shot Stopping, PSxG), (keepers, Launched, Cmp), (keepers, Launched, Att), (keepers, Launched, Cmp%), (keepers, Passes, Att (GK)), (keepers, Passes, Thr), (keepers, Passes, Launch%), (keepers, Passes, AvgLen), (keepers, Goal Kicks, Att), (keepers, Goal Kicks, Launch%), (keepers, Goal Kicks, AvgLen), (keepers, Crosses, Opp), (keepers, Crosses, Stp), (keepers, Crosses, Stp%), (keepers, Sweeper, #OPA), (keepers, Sweeper, AvgDist), (keepers, game_id, Unnamed: 28_level_1), (misc, jersey_number, Unnamed: 5_level_1), (misc, nation, Unnamed: 6_level_1), (misc, pos, Unnamed: 7_level_1), (misc, age, Unnamed: 8_level_1), (misc, Performance, min), (misc, Performance, CrdY), (misc, Performance, CrdR), (misc, Performance, 2CrdY), (misc, Performance, Fls), (misc, Performance, Fld), (misc, Performance, Off), (misc, Performance, Crs), (misc, Performance, Int), (misc, Performance, TklW), (misc, Performance, PKwon), (misc, Performance, PKcon), (misc, Performance, OG), (misc, Performance, Recov), (misc, Aerial Duels, Won), (misc, Aerial Duels, Lost), (misc, Aerial Duels, Won%), (misc, game_id, Unnamed: 26_level_1), (passing, jersey_number, Unnamed: 5_level_1), ...]
Index: []

[0 rows x 176 columns]

In [159]:
# Смотрим дубликаты столбцов
duplicates_cols = merged.T.duplicated()
duplicates_cols

summary     jersey_number  Unnamed: 5_level_1     False
            nation         Unnamed: 6_level_1     False
            pos            Unnamed: 7_level_1     False
            age            Unnamed: 8_level_1     False
            min            Unnamed: 9_level_1     False
                                                  ...  
possession  Carries        Mis                    False
                           Dis                    False
            Receiving      Rec                    False
                           PrgR                   False
            game_id        Unnamed: 32_level_1     True
Length: 176, dtype: bool

In [ ]:
# Какие столбцы повторились
merged.columns[duplicates_cols == True]

MultiIndex([(      'defense',       'Tackles',                 'Tkl'),
            (      'defense',        'Blocks',              'Blocks'),
            (         'misc', 'jersey_number',  'Unnamed: 5_level_1'),
            (         'misc',        'nation',  'Unnamed: 6_level_1'),
            (         'misc',           'pos',  'Unnamed: 7_level_1'),
            (         'misc',           'age',  'Unnamed: 8_level_1'),
            (         'misc',   'Performance',                 'min'),
            (         'misc',   'Performance',                 'Int'),
            (         'misc',   'Performance',                'TklW'),
            (         'misc',       'game_id', 'Unnamed: 26_level_1'),
            (      'passing', 'jersey_number',  'Unnamed: 5_level_1'),
            (      'passing',        'nation',  'Unnamed: 6_level_1'),
            (      'passing',           'pos',  'Unnamed: 7_level_1'),
            (      'passing',           'age',  'Unnamed: 8_level_1'),
      

In [ ]:
merged.loc[:, ('summary', 'Expected', 'xG')] != merged.loc[:, ('summary', 'Expected', 'xG')]

### Разбиение на тестовую и обучаемую выборки (в нашем случае берём мало данных, чтобы протестировать наши функции)

In [ ]:
step = len(df) // 5  # Если хотим примерно равное количество записей в каждой группе

# Выбираем первые 80%
df_train = df.iloc[:len(df)*4//5]

# Берём оставшиеся 20%
df_test = df.iloc[len(df)*4//5:]

print("Размер Train:", len(df_train))
print("Размер Test:", len(df_test))